# 0. Initialisation

In [1]:
import pandas as pd
import numpy as np 
from helpers import s3_connection

In [2]:
s3 = s3_connection()

Connection successful


# 1. Ouverture des fichiers

## 1.1. DPE

In [3]:
path_logements_existants = "clichere/diffusion/DPE/DPE_ADEME/dpe-v2-logements-existants.csv"
DPE_logements_existants = s3.read_file_from_s3(path_logements_existants)

/home/onyxia/work/STATAPP/helpers.py:33: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support sep=None with delim_whitespace=False; you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(file_in, usecols=columns_to_select, dtype=dtype_spec, sep=sep)


In [4]:
path_logements_neufs = "clichere/diffusion/DPE/DPE_ADEME/dpe-v2-logements-neufs.csv"
DPE_logements_neufs = s3.read_file_from_s3(path_logements_neufs)

/home/onyxia/work/STATAPP/helpers.py:33: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support sep=None with delim_whitespace=False; you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(file_in, usecols=columns_to_select, dtype=dtype_spec, sep=sep)


In [5]:
print(len(DPE_logements_existants.columns))
print(len(DPE_logements_neufs.columns))
print(DPE_logements_existants.columns)
print(DPE_logements_existants.columns)

33
30
Index(['Date_réception_DPE', 'Date_établissement_DPE', 'Modèle_DPE',
       'Date_fin_validité_DPE', 'Version_DPE', 'Méthode_application_DPE',
       'Etiquette_DPE', 'Etiquette_GES', 'Année_construction', 'Type_bâtiment',
       'Période_construction', 'Surface_habitable_logement', 'Adresse_brute',
       'Nom__commune_(BAN)', 'Code_INSEE_(BAN)', 'N°_voie_(BAN)',
       'Identifiant__BAN', 'Adresse_(BAN)', 'Code_postal_(BAN)', 'Score_BAN',
       'Nom__rue_(BAN)', 'Coordonnée_cartographique_X_(BAN)',
       'Coordonnée_cartographique_Y_(BAN)', 'Code_postal_(brut)',
       'N°_étage_appartement', 'Nom_résidence', 'Cage_d'escalier',
       'Complément_d'adresse_logement', 'Statut_géocodage',
       'Nom__commune_(Brut)', 'N°_département_(BAN)', 'N°_région_(BAN)',
       'Complément_d'adresse_bâtiment'],
      dtype='object')
Index(['Date_réception_DPE', 'Date_établissement_DPE', 'Modèle_DPE',
       'Date_fin_validité_DPE', 'Version_DPE', 'Méthode_application_DPE',
       'Etiquet

Il y a trois variables en plus dans le fichier logements : Année_construction, Période_construction et Type_bâtiment

In [6]:
DPE_logements_neufs["Année_construction"]='2024'
DPE_logements_neufs["Période_construction"]="après 2024"
DPE_logements_neufs["Type_bâtiment"]="NA"

In [7]:
DPE_logements = pd.concat([DPE_logements_existants, DPE_logements_neufs], ignore_index=True)

In [8]:
DPE_logements['Année_construction'] = pd.to_numeric(DPE_logements['Année_construction'], errors='coerce')

## 1.2. Valeurs foncières

### Téléchargement et fusion des bases

In [9]:
path_valeursfoncieres_2022 = "clichere/diffusion/Valeursfoncières/valeursfoncieres-2022.txt"
vf2022 = s3.read_file_from_s3(path_valeursfoncieres_2022, sep="|")

/home/onyxia/work/STATAPP/helpers.py:33: DtypeWarning: Columns (18,23,24,26,28,29,31,33,41) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_in, usecols=columns_to_select, dtype=dtype_spec, sep=sep)


In [10]:
path_valeursfoncieres_2022 = "clichere/diffusion/Valeursfoncières/valeursfoncieres-2022.txt"
vf2022 = s3.read_file_from_s3(path_valeursfoncieres_2022, sep="|")

path_valeursfoncieres_2021 = "clichere/diffusion/Valeursfoncières/valeursfoncieres-2021.txt"
vf2021 = s3.read_file_from_s3(path_valeursfoncieres_2021, sep="|")

path_valeursfoncieres_2020 = "clichere/diffusion/Valeursfoncières/valeursfoncieres-2020.txt"
vf2020 = s3.read_file_from_s3(path_valeursfoncieres_2020, sep="|")

path_valeursfoncieres_2019 = "clichere/diffusion/Valeursfoncières/valeursfoncieres-2019.txt"
vf2019 = s3.read_file_from_s3(path_valeursfoncieres_2019, sep="|")

path_valeursfoncieres_2018 = "clichere/diffusion/Valeursfoncières/valeursfoncieres-2018.txt"
vf2018 = s3.read_file_from_s3(path_valeursfoncieres_2018, sep="|")

path_valeursfoncieres_2017 = "clichere/diffusion/Valeursfoncières/valeursfoncieres-2017.txt"
vf2017 = s3.read_file_from_s3(path_valeursfoncieres_2017, sep="|")

path_valeursfoncieres_2016 = "clichere/diffusion/Valeursfoncières/valeursfoncieres-2016.txt"
vf2016 = s3.read_file_from_s3(path_valeursfoncieres_2016, sep="|")

path_valeursfoncieres_2015 = "clichere/diffusion/Valeursfoncières/valeursfoncieres-2015.txt"
vf2015 = s3.read_file_from_s3(path_valeursfoncieres_2015, sep="|")

path_valeursfoncieres_2014 = "clichere/diffusion/Valeursfoncières/valeursfoncieres-2014.txt"
vf2014 = s3.read_file_from_s3(path_valeursfoncieres_2014, sep="|")


/home/onyxia/work/STATAPP/helpers.py:33: DtypeWarning: Columns (18,23,24,26,28,29,31,33,41) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_in, usecols=columns_to_select, dtype=dtype_spec, sep=sep)
/home/onyxia/work/STATAPP/helpers.py:33: DtypeWarning: Columns (18,23,24,26,28,29,30,31,33,41) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_in, usecols=columns_to_select, dtype=dtype_spec, sep=sep)
/home/onyxia/work/STATAPP/helpers.py:33: DtypeWarning: Columns (18,23,24,26,28,31,33,41) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_in, usecols=columns_to_select, dtype=dtype_spec, sep=sep)
/home/onyxia/work/STATAPP/helpers.py:33: DtypeWarning: Columns (18,23,24,26,28,31,33,41) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_in, usecols=columns_to_select, dtype=dtype_spec, sep=sep)
/home/onyxi

In [11]:
print(vf2022.columns)

Index(['Identifiant de document', 'Reference document', '1 Articles CGI',
       '2 Articles CGI', '3 Articles CGI', '4 Articles CGI', '5 Articles CGI',
       'No disposition', 'Date mutation', 'Nature mutation', 'Valeur fonciere',
       'No voie', 'B/T/Q', 'Type de voie', 'Code voie', 'Voie', 'Code postal',
       'Commune', 'Code departement', 'Code commune', 'Prefixe de section',
       'Section', 'No plan', 'No Volume', '1er lot',
       'Surface Carrez du 1er lot', '2eme lot', 'Surface Carrez du 2eme lot',
       '3eme lot', 'Surface Carrez du 3eme lot', '4eme lot',
       'Surface Carrez du 4eme lot', '5eme lot', 'Surface Carrez du 5eme lot',
       'Nombre de lots', 'Code type local', 'Type local', 'Identifiant local',
       'Surface reelle bati', 'Nombre pieces principales', 'Nature culture',
       'Nature culture speciale', 'Surface terrain'],
      dtype='object')


In [12]:
vf = pd.concat([vf2022, vf2021, vf2020, vf2019, vf2018, vf2017, vf2016, vf2015, vf2014], ignore_index=True)


### Normalisation des adresses

In [ ]:
# Convertir explicitement toutes les colonnes en chaînes
vf['Adresse'] = vf['No voie'].apply(lambda x: str(int(x)) if pd.notna(x) else '').astype(str) + " " + \
                vf['Type de voie'].fillna('').astype(str) + " " + \
                vf['Voie'].fillna('').astype(str) + ", " + \
                vf['Code postal'].apply(lambda x: str(int(x)) if pd.notna(x) else '').astype(str) + " " + \
                vf['Commune'].fillna('').astype(str)
print(vf.describe)

In [ ]:
#vf['Adresse'] = vf['Adresse'].str.strip().replace(r'^\s*$', None, regex=True)  # Supprime les adresses vides
#print(vf2022[['No voie', 'Type de voie', 'Voie', 'Code postal', 'Commune', 'Adresse']].head())

In [14]:
#vf2022['Adresse_Normalisee'] = vf2022['Adresse'].apply(s3.normalize_address)

#print(vf2022[['Adresse', 'Adresse_Normalisee']].head())

TypeError: s3_connection.normalize_address() takes 1 positional argument but 2 were given